Cell 1: Install and Setup OpenAI API Key

This cell loads the OpenAI API key from a .env file using the dotenv library and sets up OpenAI authentication. It is required to make API calls.

In [ ]:
#!pip install python-dotenv
#!pip install openai

import os
from dotenv import load_dotenv, find_dotenv
import openai

# Find the .env file and load the environment variables
load_dotenv(find_dotenv())

# Access the OpenAI API key from the environment
openai_api_key = os.getenv('OPENAI_API_KEY')

# Ensure the key is found
if openai_api_key is None:
    raise ValueError("API key not found. Check your .env file or environment variables.")

# Set the API key for OpenAI
openai.api_key = openai_api_key

# Now you can use the OpenAI API
response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say this is a test",
    max_tokens=5
)

print(response.choices[0].text.strip())


Cell 2: Model Deprecation Check

This cell dynamically selects the correct OpenAI model based on the current date. After June 12, 2024, it switches from gpt-3.5-turbo-0301 to gpt-3.5-turbo to handle API model deprecations.

In [ ]:
# Purpose: Account for model deprecation and switch models after June 12, 2024.

import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should change to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Conditional check to switch model
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


Cell 3: Function to Call OpenAI API


This function sends a user-provided prompt to the OpenAI API using the specified model and returns the generated response. The temperature is set to 0 for consistent outputs.



In [ ]:
# Purpose: A function to directly call OpenAI's ChatCompletion API for generating completions.

def get_completion(prompt, model=llm_model):
    # Set the user message for the model
    messages = [{"role": "user", "content": prompt}]
    
    # Send the prompt to the OpenAI API and get a response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Low temperature for deterministic results
    )
    # Return the model's response
    return response.choices[0].message["content"]


Cell 4: Simple API Example

This cell demonstrates how to call the get_completion function by passing a simple math question (What is 1+1?) to the OpenAI API and getting the response.

In [ ]:
# Example: Calling the function to get a simple completion
get_completion("What is 1+1?")


Cell 5: Prompt to Translate Customer Email Using OpenAI API

This cell formats and sends a customer complaint written in pirate-style English to the OpenAI API, requesting a translation into calm and respectful American English.



# Purpose: Demonstrate how to format a prompt to translate text into a different style

# Define the text and style
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English \
in a calm and respectful tone
"""

# Create a formatted prompt by inserting the text and style
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

# Print the prompt and get a completion
print(prompt)
response = get_completion(prompt)
response


Cell 6: Install LangChain

This cell installs the LangChain library, which is used for building applications that involve chaining large language models (LLMs).

In [ ]:
# Purpose: Install LangChain library to interact with language models
# !pip install --upgrade langchain


Cell 7: Initialize LangChain Model


This cell sets up a LangChain model (ChatOpenAI) for interacting with OpenAI's GPT models. The temperature=0.0 ensures low randomness and deterministic output

In [ ]:
# Purpose: Demonstrate how to use LangChain for model interaction

# Import the necessary class for ChatOpenAI from LangChain
from langchain.chat_models import ChatOpenAI

# Initialize the model with the desired temperature (randomness control)
chat = ChatOpenAI(temperature=0.0, model=llm_model)  # Same model and deterministic behavior (temp=0)
chat


Cell 8: Define LangChain Prompt Template

This cell creates a prompt template using LangChain's ChatPromptTemplate. It is used to generate messages dynamically by filling in placeholders like {style} and {text}.



In [ ]:
# Purpose: Create a structured prompt using LangChain's ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# Import the prompt template class from LangChain
from langchain.prompts import ChatPromptTemplate

# Create the prompt template
prompt_template = ChatPromptTemplate.from_template(template_string)

# Check the prompt structure and variables
print(prompt_template.messages[0].prompt.input_variables)


Cell 9: Format LangChain Messages


This cell formats a customer message by applying the predefined LangChain prompt template, generating a message that will be sent to the model.

In [ ]:
# Define the customer style and email
customer_style = """American English in a calm and respectful tone"""
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! 
And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. 
I need yer help right now, matey!
"""

# Format the prompt using the template
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

# Display the customer message object and the first message content
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])


Cell 10: Call LangChain Model to Generate Response

This cell sends the formatted customer message to the LangChain model and prints the model's response (translated into a calm and respectful tone).

In [ ]:
# Call the model to translate the message using the defined style
customer_response = chat(customer_messages)

# Output the model's response
print(customer_response.content)


Cell 11: Translate Service Reply into Pirate English


This cell translates a service reply into Pirate English using LangChain's model by formatting the message and calling the model.



In [ ]:
# Purpose: Translating another text (service reply) to Pirate English

# Define a service reply and style to translate into Pirate English
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!"""
service_style_pirate = "a polite tone that speaks in English Pirate"

# Format the service message
service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)

# Print the formatted service message
print(service_messages[0].content)

# Get the translation response from the model
service_response = chat(service_messages)
print(service_response.content)


Cell 12: Extract Structured Data from a Customer Review

This cell extracts structured information (gift status, delivery days, and price comments) from an unstructured customer review using LangChain's template and model.



In [ ]:
# Purpose: Extract specific structured data (gift, delivery days, price) from an unstructured review

customer_review = """\
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's anniversary present. 
I think my wife liked it so much she was speechless. 
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. 
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

# Define the review template
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

Format the output as JSON with the following keys: gift, delivery_days, price_value

text: {text}
"""

# Create the prompt template from the review template
prompt_template = ChatPromptTemplate.from_template(review_template)

# Format the message with customer review
messages = prompt_template.format_messages(text=customer_review)

# Call the model to get structured output
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)

# Print the model's response
print(response.content)


Cell 13: Parse Model Output into a Dictionary


This cell defines schemas for the data we want to extract (gift status, delivery days, price value) and generates format instructions for parsing the model's output into a structured Python dictionary.

In [ ]:
# Purpose: Parse the LLM output into a structured Python dictionary

# Import necessary parsers from LangChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Define response schemas for each piece of data we want to extract
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

# Combine all schemas into a list
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

# Create a StructuredOutputParser using the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get format instructions for the parser
format_instructions = output_parser.get_format_instructions()

# Print format instructions to see the expected structure
print(format_instructions)


Cell 14: Apply Output Parser to Extract Data

This cell combines the output parser's format instructions with the review extraction template. It sends the prompt to the LangChain model and parses the model’s output into a structured Python dictionary.

In [ ]:
# Purpose: Integrate format instructions into the review extraction prompt

# Update the review template with format instructions
review_template_2 = f"""\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {{text}}

{format_instructions}
"""

# Create the prompt with the updated format instructions
prompt = ChatPromptTemplate.from_template(template=review_template_2)

# Format the message with review and format instructions
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

# Print the message to be sent to the model
print(messages[0].content)

# Get the model's response
response = chat(messages)
print(response.content)

# Parse the response into a Python dictionary
output_dict = output_parser.parse(response.content)

# Check the output dictionary and the value for "delivery_days"
output_dict
output_dict.get('delivery_days')
